# طبقه‌بندی زباله با شبکه عصبی کانولوشنی (CNN)
## Waste Classification using Custom CNN

این نوت‌بوک یک مدل CNN سفارشی (بدون استفاده از Transfer Learning) برای طبقه‌بندی زباله‌ها به **چهار دسته** می‌سازد:

| کلاس | Class | توضیح |
|-------|-------|-------|
| پلاستیک | Plastic | بطری، کیسه، ظروف پلاستیکی |
| کاغذ | Paper | روزنامه، مقوا، اسناد |
| فلز | Metal | قوطی، فویل، ظروف فلزی |
| آلی | Organic | پسماند غذایی، گیاهان، مواد تجزیه‌پذیر |

### ویژگی‌ها:
- مدل CNN سفارشی از صفر (بدون مدل از پیش آموزش‌دیده)
- دیتاست TrashNet از GitHub
- Data Augmentation قوی
- ارزیابی کامل با Confusion Matrix و Classification Report
- اپلیکیشن وب Flask با رابط فارسی برای تست با دوربین گوشی

> **نکته:** برای آموزش سریع‌تر، از Runtime > Change runtime type > GPU استفاده کنید.

---
## ۱. نصب کتابخانه‌ها و بررسی GPU

In [ ]:
# وارد کردن کتابخانه‌ها
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import shutil
import glob
import json
import random
import warnings
warnings.filterwarnings('ignore')

# بررسی نسخه تنسورفلو و GPU
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# فعال‌سازی رشد حافظه GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("GPU memory growth enabled")

---
## ۲. دانلود دیتاست TrashNet از GitHub

دیتاست TrashNet شامل تصاویر زباله در ۶ دسته است. ما آن را مستقیماً از GitHub دانلود می‌کنیم (بدون نیاز به Kaggle API).

In [ ]:
# پاک کردن فایل‌های قبلی
!rm -rf dataset-resized* waste_dataset 2>/dev/null

# دانلود مستقیم دیتاست از GitHub
!wget -q https://github.com/garythung/trashnet/raw/master/data/dataset-resized.zip -O dataset-resized.zip

# استخراج فایل zip
!unzip -q dataset-resized.zip

# بررسی محتویات
!ls -la dataset-resized/

In [ ]:
# تنظیم مسیر دیتاست
if os.path.exists('dataset-resized'):
    dataset_path = 'dataset-resized'
else:
    for root, dirs, files in os.walk('.'):
        if 'plastic' in dirs and 'paper' in dirs:
            dataset_path = root
            break
    else:
        raise FileNotFoundError("دیتاست پیدا نشد!")

print(f"مسیر دیتاست: {dataset_path}")

# بررسی ساختار دیتاست و تعداد تصاویر
print("\n=== ساختار دیتاست ===")
total_images = 0
for folder in sorted(os.listdir(dataset_path)):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        count = len([f for f in os.listdir(folder_path)
                     if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        total_images += count
        print(f"  {folder}: {count} تصویر")
print(f"\nمجموع: {total_images} تصویر")

---
## ۳. آماده‌سازی و تقسیم داده‌ها

دیتاست TrashNet شامل ۶ کلاس است. ما آن‌ها را به ۴ کلاس نگاشت می‌کنیم:

| کلاس اصلی | کلاس هدف | توضیح |
|------------|----------|-------|
| plastic | plastic | پلاستیک |
| paper | paper | کاغذ |
| metal | metal | فلز |
| cardboard | organic | مقوا → آلی |
| trash | organic | زباله عمومی → آلی |
| glass | - | حذف می‌شود |

In [ ]:
# ایجاد پوشه‌های جدید برای ۴ کلاس
new_dataset_path = 'waste_dataset'
categories = ['plastic', 'paper', 'metal', 'organic']

# پاک کردن پوشه قبلی اگر وجود دارد
if os.path.exists(new_dataset_path):
    shutil.rmtree(new_dataset_path)

# ایجاد ساختار پوشه: train, val, test
for split in ['train', 'val', 'test']:
    for cat in categories:
        os.makedirs(os.path.join(new_dataset_path, split, cat), exist_ok=True)

# نگاشت کلاس‌های TrashNet به ۴ کلاس ما
class_mapping = {
    'plastic': 'plastic',
    'paper': 'paper',
    'metal': 'metal',
    'cardboard': 'organic',
    'trash': 'organic'
    # glass حذف می‌شود چون در ۴ کلاس ما نیست
}

# جمع‌آوری و تقسیم داده‌ها
for original_class, target_class in class_mapping.items():
    source_folder = os.path.join(dataset_path, original_class)
    if not os.path.exists(source_folder):
        print(f"پوشه '{original_class}' پیدا نشد، رد شد...")
        continue

    # پیدا کردن همه تصاویر
    images = glob.glob(os.path.join(source_folder, '*.jpg')) + \
             glob.glob(os.path.join(source_folder, '*.jpeg')) + \
             glob.glob(os.path.join(source_folder, '*.png'))

    if len(images) == 0:
        print(f"تصویری در '{original_class}' پیدا نشد")
        continue

    # تقسیم: 70% train, 15% val, 15% test
    train_imgs, temp_imgs = train_test_split(images, test_size=0.3, random_state=42)
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

    # کپی فایل‌ها با نام یکتا
    for i, img in enumerate(train_imgs):
        ext = os.path.splitext(img)[1]
        shutil.copy(img, os.path.join(new_dataset_path, 'train', target_class, f"{original_class}_{i}{ext}"))

    for i, img in enumerate(val_imgs):
        ext = os.path.splitext(img)[1]
        shutil.copy(img, os.path.join(new_dataset_path, 'val', target_class, f"{original_class}_{i}{ext}"))

    for i, img in enumerate(test_imgs):
        ext = os.path.splitext(img)[1]
        shutil.copy(img, os.path.join(new_dataset_path, 'test', target_class, f"{original_class}_{i}{ext}"))

    print(f"{original_class} -> {target_class}: train={len(train_imgs)}, val={len(val_imgs)}, test={len(test_imgs)}")

In [ ]:
# آمار نهایی دیتاست
print("=" * 50)
print("آمار نهایی دیتاست")
print("=" * 50)
for split in ['train', 'val', 'test']:
    print(f"\n{split.upper()}:")
    total = 0
    for cat in categories:
        path = os.path.join(new_dataset_path, split, cat)
        count = len(os.listdir(path)) if os.path.exists(path) else 0
        total += count
        print(f"  {cat}: {count} تصویر")
    print(f"  مجموع: {total} تصویر")

---
## ۴. افزایش داده (Data Augmentation) و ایجاد Generator

In [ ]:
# پارامترها
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data Augmentation برای آموزش
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# فقط نرمال‌سازی برای اعتبارسنجی و تست
val_test_datagen = ImageDataGenerator(rescale=1./255)

# ایجاد Generator ها
train_generator = train_datagen.flow_from_directory(
    os.path.join(new_dataset_path, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    os.path.join(new_dataset_path, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(new_dataset_path, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# نمایش اطلاعات کلاس‌ها
class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)
print(f"\nکلاس‌ها: {class_names}")
print(f"تعداد کلاس‌ها: {num_classes}")
print(f"اندیس کلاس‌ها: {train_generator.class_indices}")

In [ ]:
# نمایش نمونه تصاویر افزایش‌یافته
images, labels = next(train_generator)

plt.figure(figsize=(16, 8))
for i in range(min(8, len(images))):
    plt.subplot(2, 4, i + 1)
    plt.imshow(images[i])
    label_idx = np.argmax(labels[i])
    plt.title(f'{class_names[label_idx]}', fontsize=12)
    plt.axis('off')
plt.suptitle('نمونه تصاویر آموزشی (با Data Augmentation)', fontsize=14)
plt.tight_layout()
plt.show()

---
## ۵. ساخت مدل CNN سفارشی

مدل CNN ما شامل ۵ بلوک کانولوشنی با فیلترهای افزایشی (۳۲ → ۶۴ → ۱۲۸ → ۲۵۶ → ۵۱۲) است.

### معماری مدل:
- **بلوک‌های کانولوشنی:** هر بلوک شامل دو لایه Conv2D + BatchNormalization + ReLU + MaxPooling + Dropout
- **Global Average Pooling:** به جای Flatten برای کاهش پارامترها
- **لایه‌های Dense:** ۵۱۲ → ۲۵۶ → ۴ (softmax)
- **Regularization:** L2 روی تمام لایه‌ها + Dropout + BatchNormalization

In [ ]:
def build_cnn_model(input_shape=(224, 224, 3), num_classes=4):
    """
    ساخت مدل CNN سفارشی برای طبقه‌بندی زباله
    این مدل از صفر ساخته شده و از هیچ مدل از پیش آموزش‌دیده‌ای استفاده نمی‌کند
    """
    model = models.Sequential([
        # === بلوک ۱: ۳۲ فیلتر ===
        layers.Conv2D(32, (3, 3), padding='same', kernel_regularizer=l2(0.001),
                      input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(32, (3, 3), padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # === بلوک ۲: ۶۴ فیلتر ===
        layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # === بلوک ۳: ۱۲۸ فیلتر ===
        layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # === بلوک ۴: ۲۵۶ فیلتر ===
        layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # === بلوک ۵: ۵۱۲ فیلتر ===
        layers.Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # === Global Average Pooling ===
        layers.GlobalAveragePooling2D(),

        # === لایه‌های Fully Connected ===
        layers.Dense(512, kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.5),

        layers.Dense(256, kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.3),

        # === لایه خروجی ===
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

# ساخت مدل
model = build_cnn_model(num_classes=num_classes)
model.summary()

In [ ]:
# کامپایل مدل
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("مدل با موفقیت کامپایل شد!")
print(f"تعداد کل پارامترها: {model.count_params():,}")

---
## ۶. آموزش مدل

از Callback های زیر استفاده می‌کنیم:
- **EarlyStopping:** توقف زودهنگام در صورت عدم بهبود (صبر: ۱۵ epoch)
- **ModelCheckpoint:** ذخیره بهترین مدل بر اساس دقت اعتبارسنجی
- **ReduceLROnPlateau:** کاهش نرخ یادگیری در صورت عدم بهبود

In [ ]:
# تعریف Callback ها
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_waste_classifier.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# آموزش مدل
EPOCHS = 50

print("شروع آموزش...")
print(f"تعداد نمونه‌های آموزشی: {train_generator.samples}")
print(f"تعداد نمونه‌های اعتبارسنجی: {val_generator.samples}")
print(f"تعداد epoch ها: {EPOCHS}")
print("=" * 50)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

---
## ۷. نمایش نتایج آموزش (نمودار Accuracy و Loss)

In [ ]:
# رسم نمودار Accuracy و Loss
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# نمودار دقت
axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2, color='blue')
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2, color='orange')
axes[0].set_title('دقت مدل (Accuracy)', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# نمودار خطا
axes[1].plot(history.history['loss'], label='Train Loss', linewidth=2, color='blue')
axes[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2, color='orange')
axes[1].set_title('خطای مدل (Loss)', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150)
plt.show()

# بهترین نتایج
best_val_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1
print(f"\nبهترین دقت اعتبارسنجی: {best_val_acc*100:.2f}% در epoch {best_epoch}")

---
## ۸. ارزیابی مدل روی داده‌های تست

In [ ]:
# بارگذاری بهترین مدل
best_model = keras.models.load_model('best_waste_classifier.keras')
print("بهترین مدل بارگذاری شد!")

# ارزیابی روی داده تست
print("\nارزیابی روی داده‌های تست...")
test_loss, test_accuracy = best_model.evaluate(test_generator, verbose=1)
print(f"\n{'='*50}")
print(f"دقت تست: {test_accuracy*100:.2f}%")
print(f"خطای تست: {test_loss:.4f}")
print(f"{'='*50}")

# پیش‌بینی روی داده‌های تست
test_generator.reset()
predictions = best_model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# گزارش طبقه‌بندی
print(f"\n{'='*50}")
print("گزارش طبقه‌بندی (Classification Report)")
print(f"{'='*50}")
print(classification_report(true_classes, predicted_classes, target_names=class_names))

In [ ]:
# ماتریس درهم‌ریختگی (Confusion Matrix)
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            annot_kws={'size': 14})
plt.title('ماتریس درهم‌ریختگی (Confusion Matrix)', fontsize=16)
plt.xlabel('برچسب پیش‌بینی شده', fontsize=12)
plt.ylabel('برچسب واقعی', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

# دقت هر کلاس
print("\nدقت هر کلاس:")
for i, name in enumerate(class_names):
    class_acc = cm[i, i] / cm[i].sum() * 100 if cm[i].sum() > 0 else 0
    print(f"  {name}: {class_acc:.2f}%")

---
## ۹. نمایش پیش‌بینی‌های مدل

In [ ]:
# نمایش تصاویر با پیش‌بینی و برچسب واقعی
test_generator.reset()
images, labels = next(test_generator)
preds = best_model.predict(images, verbose=0)

plt.figure(figsize=(16, 12))
for i in range(min(12, len(images))):
    plt.subplot(3, 4, i + 1)
    plt.imshow(images[i])

    true_label = class_names[np.argmax(labels[i])]
    pred_label = class_names[np.argmax(preds[i])]
    confidence = np.max(preds[i]) * 100

    color = 'green' if true_label == pred_label else 'red'
    plt.title(f'True: {true_label}\nPred: {pred_label} ({confidence:.1f}%)',
              color=color, fontsize=10)
    plt.axis('off')

plt.suptitle('پیش‌بینی‌های مدل (سبز=صحیح، قرمز=غلط)', fontsize=14)
plt.tight_layout()
plt.savefig('predictions.png', dpi=150)
plt.show()

---
## ۱۰. پیش‌بینی تصویر جدید

تابع زیر یک تصویر دلخواه را دریافت کرده، نوع زباله را تشخیص می‌دهد و نتایج را با نمودار احتمالات نمایش می‌دهد.

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_single_image(model, img_path, class_names, img_size=(224, 224)):
    """پیش‌بینی کلاس یک تصویر"""
    # بارگذاری و پیش‌پردازش تصویر
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # پیش‌بینی
    predictions = model.predict(img_array, verbose=0)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = np.max(predictions[0]) * 100

    # نمایش نتایج
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # تصویر
    axes[0].imshow(img)
    axes[0].set_title(f'پیش‌بینی: {predicted_class}\nاطمینان: {confidence:.2f}%', fontsize=14)
    axes[0].axis('off')

    # احتمالات
    colors = ['#FF6B6B' if i == np.argmax(predictions[0]) else '#4ECDC4'
              for i in range(len(class_names))]
    bars = axes[1].barh(class_names, predictions[0] * 100, color=colors)
    axes[1].set_xlabel('احتمال (%)', fontsize=12)
    axes[1].set_title('احتمال هر کلاس', fontsize=14)
    axes[1].set_xlim(0, 100)

    for bar, prob in zip(bars, predictions[0]):
        axes[1].text(prob * 100 + 1, bar.get_y() + bar.get_height() / 2,
                     f'{prob*100:.1f}%', va='center', fontsize=10)

    plt.tight_layout()
    plt.show()

    return predicted_class, predictions[0]

# تست با نمونه‌هایی از هر کلاس
print("تست با تصاویر نمونه از هر کلاس:\n")
for cat in class_names:
    test_folder = os.path.join(new_dataset_path, 'test', cat)
    if os.path.exists(test_folder):
        test_images = [f for f in os.listdir(test_folder)
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if test_images:
            sample_img = random.choice(test_images)
            img_path = os.path.join(test_folder, sample_img)
            print(f"\n{'='*50}")
            print(f"تست تصویر {cat.upper()}: {sample_img}")
            print(f"{'='*50}")
            pred_class, probs = predict_single_image(best_model, img_path, class_names)

In [ ]:
# آپلود تصویر سفارشی برای پیش‌بینی (فقط در Google Colab)
try:
    from google.colab import files

    print("یک تصویر آپلود کنید:")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"\n{'='*50}")
        print(f"طبقه‌بندی: {filename}")
        print(f"{'='*50}")

        pred_class, probs = predict_single_image(best_model, filename, class_names)

        print(f"\nنتیجه: {pred_class}")
        print("\nاحتمالات:")
        for name, prob in zip(class_names, probs):
            print(f"  {name}: {prob*100:.2f}%")

except ImportError:
    print("این سلول فقط در Google Colab کار می‌کند.")
    print("برای تست با تصویر سفارشی از تابع predict_single_image استفاده کنید.")

---
## ۱۱. ذخیره و دانلود مدل

In [ ]:
# ذخیره مدل نهایی
best_model.save('waste_classifier_final.keras')
print("مدل ذخیره شد: waste_classifier_final.keras")

# ذخیره نام کلاس‌ها
with open('class_names.json', 'w') as f:
    json.dump(class_names, f, indent=2)
print("نام کلاس‌ها ذخیره شد: class_names.json")

# ذخیره تاریخچه آموزش
history_dict = {
    'accuracy': [float(x) for x in history.history['accuracy']],
    'val_accuracy': [float(x) for x in history.history['val_accuracy']],
    'loss': [float(x) for x in history.history['loss']],
    'val_loss': [float(x) for x in history.history['val_loss']]
}
with open('training_history.json', 'w') as f:
    json.dump(history_dict, f, indent=2)
print("تاریخچه آموزش ذخیره شد: training_history.json")

# دانلود فایل‌ها (فقط در Colab)
try:
    from google.colab import files as colab_files
    print("\nدانلود فایل‌ها...")
    for filename in ['waste_classifier_final.keras', 'best_waste_classifier.keras',
                     'class_names.json', 'training_history.json',
                     'training_history.png', 'confusion_matrix.png']:
        if os.path.exists(filename):
            colab_files.download(filename)
            print(f"  دانلود شد: {filename}")
except ImportError:
    print("دانلود خودکار فقط در Colab کار می‌کند.")

---
## ۱۲. اپلیکیشن وب با Flask و دوربین گوشی

این بخش یک سرور وب Flask راه‌اندازی می‌کند. با استفاده از ngrok یک لینک عمومی ایجاد می‌شود که می‌توانید از گوشی موبایل به آن وصل شوید و از دوربین گوشی برای تشخیص نوع زباله استفاده کنید.

### مراحل:
1. نصب Flask و pyngrok
2. ایجاد فایل `app.py` (سرور Flask)
3. ایجاد فایل `templates/index.html` (رابط کاربری فارسی)
4. اجرای سرور و ایجاد تونل ngrok

In [ ]:
# نصب کتابخانه‌های مورد نیاز
!pip install -q flask pyngrok

In [ ]:
%%writefile app.py
"""
Waste Classification Web Application
اپلیکیشن وب طبقه‌بندی زباله با استفاده از دوربین گوشی
"""

import os
import json
import base64
import numpy as np
from io import BytesIO
from PIL import Image
from flask import Flask, render_template, request, jsonify, send_file
import tensorflow as tf
from tensorflow import keras

app = Flask(__name__)

# متغیرهای سراسری
model = None
class_names = None
MODEL_PATH = 'waste_classifier_final.keras'
BEST_MODEL_PATH = 'best_waste_classifier.keras'
CLASS_NAMES_PATH = 'class_names.json'
IMG_SIZE = (224, 224)

# ترجمه فارسی کلاس‌ها
CLASS_NAMES_FA = {
    'plastic': 'پلاستیک',
    'paper': 'کاغذ',
    'metal': 'فلز',
    'organic': 'آلی'
}

RECYCLING_TIPS = {
    'plastic': "پلاستیک‌ها را شسته و در سطل بازیافت بیندازید.",
    'paper': "کاغذها را خشک نگه دارید و در سطل بازیافت بیندازید.",
    'metal': "قوطی‌های فلزی را شسته و در سطل بازیافت بیندازید.",
    'organic': "زباله‌های آلی را در سطل کمپوست بیندازید."
}


def load_model():
    """بارگذاری مدل و نام کلاس‌ها"""
    global model, class_names

    model_path = MODEL_PATH if os.path.exists(MODEL_PATH) else BEST_MODEL_PATH
    if os.path.exists(model_path):
        model = keras.models.load_model(model_path)
        print(f"Model loaded from {model_path}")
    else:
        print(f"Warning: Model not found at {model_path}")

    if os.path.exists(CLASS_NAMES_PATH):
        with open(CLASS_NAMES_PATH, 'r') as f:
            class_names = json.load(f)
    else:
        class_names = ['metal', 'organic', 'paper', 'plastic']


def preprocess_image(image_data):
    """پیش‌پردازش تصویر برای پیش‌بینی"""
    if ',' in image_data:
        image_data = image_data.split(',')[1]

    image_bytes = base64.b64decode(image_data)
    img = Image.open(BytesIO(image_bytes))

    if img.mode != 'RGB':
        img = img.convert('RGB')

    img = img.resize(IMG_SIZE)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        if 'image' not in data:
            return jsonify({'error': 'No image data'}), 400

        img_array = preprocess_image(data['image'])
        predictions = model.predict(img_array, verbose=0)
        predicted_idx = np.argmax(predictions[0])
        predicted_class = class_names[predicted_idx]
        confidence = float(predictions[0][predicted_idx]) * 100

        all_probs = {class_names[i]: float(predictions[0][i]) * 100
                     for i in range(len(class_names))}

        return jsonify({
            'success': True,
            'prediction': {
                'class': predicted_class,
                'class_fa': CLASS_NAMES_FA.get(predicted_class, predicted_class),
                'confidence': round(confidence, 2),
                'tip': RECYCLING_TIPS.get(predicted_class, ''),
                'all_probabilities': {k: round(v, 2) for k, v in all_probs.items()}
            }
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/download-model')
def download_model():
    model_path = MODEL_PATH if os.path.exists(MODEL_PATH) else BEST_MODEL_PATH
    if os.path.exists(model_path):
        return send_file(model_path, as_attachment=True)
    return jsonify({'error': 'Model not found'}), 404


@app.route('/status')
def status():
    return jsonify({
        'server': 'running',
        'model_loaded': model is not None,
        'class_names': class_names
    })


if __name__ == '__main__':
    load_model()
    app.run(host='0.0.0.0', port=5000, debug=False)

In [ ]:
# ایجاد پوشه templates
import os
os.makedirs('templates', exist_ok=True)

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
    <title>طبقه‌بندی زباله | Waste Classifier</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: 'Segoe UI', Tahoma, Arial, sans-serif;
            background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%);
            min-height: 100vh; color: #fff; overflow-x: hidden;
        }
        .container { max-width: 800px; margin: 0 auto; padding: 20px; }
        header { text-align: center; padding: 20px 0; margin-bottom: 20px; }
        h1 { font-size: 2rem; color: #00d4ff; margin-bottom: 10px; }
        .subtitle { color: #a0a0a0; font-size: 1rem; }
        .camera-container {
            position: relative; background: #1a1a2e; border-radius: 20px;
            overflow: hidden; box-shadow: 0 10px 40px rgba(0,0,0,0.5); margin-bottom: 20px;
        }
        #video { width: 100%; max-height: 60vh; object-fit: cover; display: block; }
        #canvas { display: none; }
        .camera-overlay {
            position: absolute; top: 0; left: 0; right: 0; bottom: 0;
            display: flex; align-items: center; justify-content: center;
            background: rgba(0,0,0,0.7); z-index: 10;
        }
        .camera-overlay.hidden { display: none; }
        .camera-overlay p { font-size: 1.2rem; color: #00d4ff; }
        .controls { display: flex; gap: 15px; justify-content: center; flex-wrap: wrap; margin-bottom: 20px; }
        .btn {
            padding: 15px 30px; font-size: 1.1rem; font-weight: bold;
            border: none; border-radius: 50px; cursor: pointer;
            transition: all 0.3s ease; display: flex; align-items: center; gap: 10px;
        }
        .btn-primary { background: linear-gradient(45deg, #00d4ff, #0099cc); color: #fff; }
        .btn-primary:hover { transform: scale(1.05); box-shadow: 0 5px 20px rgba(0,212,255,0.5); }
        .btn-secondary { background: linear-gradient(45deg, #4ecdc4, #2d9a93); color: #fff; }
        .btn:disabled { opacity: 0.5; cursor: not-allowed; }
        .result-container {
            background: rgba(255,255,255,0.1); border-radius: 20px;
            padding: 25px; margin-bottom: 20px; backdrop-filter: blur(10px); display: none;
        }
        .result-container.show { display: block; animation: fadeIn 0.5s ease; }
        @keyframes fadeIn { from { opacity: 0; transform: translateY(20px); } to { opacity: 1; transform: translateY(0); } }
        .result-header { text-align: center; margin-bottom: 20px; }
        .result-class { font-size: 2rem; color: #00d4ff; margin-bottom: 10px; }
        .result-confidence { font-size: 1.5rem; color: #4ecdc4; }
        .result-tip {
            background: rgba(78,205,196,0.2); padding: 15px; border-radius: 10px;
            margin-top: 15px; border-right: 4px solid #4ecdc4;
        }
        .probabilities { margin-top: 20px; }
        .prob-item { margin-bottom: 15px; }
        .prob-label { display: flex; justify-content: space-between; margin-bottom: 5px; font-size: 0.95rem; }
        .prob-bar { height: 12px; background: rgba(255,255,255,0.1); border-radius: 6px; overflow: hidden; }
        .prob-fill { height: 100%; border-radius: 6px; transition: width 0.5s ease; }
        .prob-fill.plastic { background: linear-gradient(90deg, #ff6b6b, #ff8e8e); }
        .prob-fill.paper { background: linear-gradient(90deg, #ffd93d, #ffed4a); }
        .prob-fill.metal { background: linear-gradient(90deg, #6c5ce7, #a29bfe); }
        .prob-fill.organic { background: linear-gradient(90deg, #00b894, #55efc4); }
        .loading { display: none; text-align: center; padding: 20px; }
        .loading.show { display: block; }
        .spinner {
            width: 50px; height: 50px; border: 4px solid rgba(0,212,255,0.2);
            border-top-color: #00d4ff; border-radius: 50%;
            animation: spin 1s linear infinite; margin: 0 auto 15px;
        }
        @keyframes spin { to { transform: rotate(360deg); } }
        .switch-camera {
            position: absolute; top: 10px; left: 10px; z-index: 15;
            background: rgba(0,0,0,0.5); border: none; color: white;
            width: 45px; height: 45px; border-radius: 50%; font-size: 1.5rem; cursor: pointer;
        }
        #captured-preview { display: none; width: 100%; max-height: 60vh; object-fit: contain; }
        #captured-preview.show { display: block; }
        footer { text-align: center; padding: 20px; color: #666; font-size: 0.9rem; }
        @media (max-width: 600px) { h1 { font-size: 1.5rem; } .btn { padding: 12px 20px; font-size: 1rem; } }
    </style>
</head>
<body>
    <div class="container">
        <header>
            <h1>طبقه‌بندی زباله</h1>
            <p class="subtitle">Waste Classification with CNN</p>
        </header>
        <div class="camera-container">
            <div class="camera-overlay" id="camera-overlay"><p>در حال باز کردن دوربین...</p></div>
            <button class="switch-camera" id="switch-camera" title="تغییر دوربین">&#x1F504;</button>
            <video id="video" autoplay playsinline></video>
            <canvas id="canvas"></canvas>
            <img id="captured-preview" alt="تصویر گرفته شده">
        </div>
        <div class="controls">
            <button class="btn btn-primary" id="capture-btn" disabled>گرفتن عکس</button>
            <button class="btn btn-secondary" id="retry-btn" style="display:none;">عکس جدید</button>
        </div>
        <div class="loading" id="loading"><div class="spinner"></div><p>در حال تحلیل تصویر...</p></div>
        <div class="result-container" id="result-container">
            <div class="result-header">
                <div class="result-class" id="result-class"></div>
                <div class="result-confidence" id="result-confidence"></div>
            </div>
            <div class="result-tip" id="result-tip"></div>
            <div class="probabilities" id="probabilities"></div>
        </div>
        <footer><p>سیستم طبقه‌بندی هوشمند زباله - ساخته شده با TensorFlow</p></footer>
    </div>
    <script>
        const video = document.getElementById('video');
        const canvas = document.getElementById('canvas');
        const capturedPreview = document.getElementById('captured-preview');
        const captureBtn = document.getElementById('capture-btn');
        const retryBtn = document.getElementById('retry-btn');
        const switchCameraBtn = document.getElementById('switch-camera');
        const cameraOverlay = document.getElementById('camera-overlay');
        const loading = document.getElementById('loading');
        const resultContainer = document.getElementById('result-container');
        let stream = null;
        let facingMode = 'environment';

        async function initCamera() {
            try {
                if (stream) stream.getTracks().forEach(track => track.stop());
                stream = await navigator.mediaDevices.getUserMedia({
                    video: { facingMode: facingMode, width: { ideal: 1280 }, height: { ideal: 720 } }
                });
                video.srcObject = stream;
                video.onloadedmetadata = () => { cameraOverlay.classList.add('hidden'); captureBtn.disabled = false; };
            } catch (err) {
                cameraOverlay.innerHTML = '<p style="color:#ff6b6b;">خطا در دسترسی به دوربین</p>';
            }
        }

        switchCameraBtn.addEventListener('click', () => {
            facingMode = facingMode === 'environment' ? 'user' : 'environment'; initCamera();
        });

        captureBtn.addEventListener('click', () => {
            canvas.width = video.videoWidth; canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            const imageData = canvas.toDataURL('image/jpeg', 0.9);
            capturedPreview.src = imageData; capturedPreview.classList.add('show');
            video.style.display = 'none'; switchCameraBtn.style.display = 'none';
            captureBtn.style.display = 'none'; retryBtn.style.display = 'flex';
            resultContainer.classList.remove('show');
            predictImage(imageData);
        });

        retryBtn.addEventListener('click', () => {
            capturedPreview.classList.remove('show'); video.style.display = 'block';
            switchCameraBtn.style.display = 'block'; captureBtn.style.display = 'flex';
            retryBtn.style.display = 'none'; resultContainer.classList.remove('show');
        });

        async function predictImage(imageData) {
            loading.classList.add('show');
            try {
                const response = await fetch('/predict', {
                    method: 'POST', headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ image: imageData })
                });
                const data = await response.json();
                loading.classList.remove('show');
                if (data.success) displayResult(data.prediction);
            } catch (err) { loading.classList.remove('show'); }
        }

        function displayResult(prediction) {
            document.getElementById('result-class').textContent = prediction.class_fa;
            document.getElementById('result-confidence').textContent = 'اطمینان: ' + prediction.confidence + '%';
            document.getElementById('result-tip').textContent = prediction.tip;
            const probsContainer = document.getElementById('probabilities');
            probsContainer.innerHTML = '<h4 style="margin-bottom:15px;">احتمالات:</h4>';
            const classLabels = { 'plastic': 'پلاستیک', 'paper': 'کاغذ', 'metal': 'فلز', 'organic': 'آلی' };
            for (const [cls, prob] of Object.entries(prediction.all_probabilities)) {
                probsContainer.innerHTML += '<div class="prob-item"><div class="prob-label"><span>' +
                    (classLabels[cls] || cls) + '</span><span>' + prob + '%</span></div>' +
                    '<div class="prob-bar"><div class="prob-fill ' + cls + '" style="width:' + prob + '%"></div></div></div>';
            }
            resultContainer.classList.add('show');
        }

        document.addEventListener('DOMContentLoaded', () => initCamera());
    </script>
</body>
</html>

### اجرای سرور با ngrok

با اجرای سلول زیر، سرور Flask راه‌اندازی شده و یک لینک عمومی ngrok ایجاد می‌شود.
لینک را در مرورگر گوشی خود باز کنید.

> **نکته:** برای استفاده طولانی‌تر از ngrok، یک حساب رایگان در [ngrok.com](https://dashboard.ngrok.com) بسازید و auth token خود را تنظیم کنید.

In [ ]:
# اجرای سرور Flask با تونل ngrok
from pyngrok import ngrok
import threading
import time

# اگر auth token دارید، خط زیر را از حالت کامنت خارج کنید:
# ngrok.set_auth_token("YOUR_AUTH_TOKEN")

def run_flask():
    """اجرای Flask در thread جداگانه"""
    import app
    app.load_model()
    app.app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# شروع Flask در پس‌زمینه
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# صبر برای راه‌اندازی سرور
time.sleep(3)

# ایجاد تونل ngrok
public_url = ngrok.connect(5000)

print("\n" + "=" * 60)
print("سرور طبقه‌بندی زباله راه‌اندازی شد!")
print("=" * 60)
print(f"\nلینک برای گوشی موبایل:")
print(f"   {public_url}")
print("\nاین لینک را در مرورگر گوشی باز کنید")
print("=" * 60)

---
## خلاصه

در این نوت‌بوک:

1. **دیتاست TrashNet** را مستقیماً از GitHub دانلود کردیم (بدون نیاز به Kaggle)
2. **داده‌ها را به ۴ کلاس** نگاشت کردیم: پلاستیک، کاغذ، فلز، آلی
3. **یک مدل CNN سفارشی** با ۵ بلوک کانولوشنی از صفر ساختیم:
   - BatchNormalization برای پایدارسازی آموزش
   - L2 Regularization برای جلوگیری از بیش‌برازش
   - Dropout در تمام بلوک‌ها
   - Global Average Pooling به جای Flatten
4. **Data Augmentation** قوی برای بهبود عملکرد
5. **مدل را آموزش دادیم** با EarlyStopping، ModelCheckpoint و ReduceLROnPlateau
6. **ارزیابی کامل:** ماتریس درهم‌ریختگی، گزارش طبقه‌بندی، دقت هر کلاس
7. **اپلیکیشن وب Flask** با رابط فارسی و دسترسی از دوربین گوشی

### معماری مدل:
```
Input (224x224x3)
  → [Conv32 → BN → ReLU] x2 → MaxPool → Dropout(0.25)
  → [Conv64 → BN → ReLU] x2 → MaxPool → Dropout(0.25)
  → [Conv128 → BN → ReLU] x2 → MaxPool → Dropout(0.25)
  → [Conv256 → BN → ReLU] x2 → MaxPool → Dropout(0.25)
  → Conv512 → BN → ReLU → GlobalAveragePooling
  → Dense(512) → BN → ReLU → Dropout(0.5)
  → Dense(256) → BN → ReLU → Dropout(0.3)
  → Dense(4, softmax)
```